In [ ]:
with open("biology.txt",'r', encoding='utf-8') as f:
    text = f.read()
    
    chars = sorted(set(text))
    print(chars)
    
    vocab_Size = len(chars)
    print(vocab_Size)
    
    
    ## building a map 
    
    stoi = {ch:i for i,ch in enumerate(chars)}
    Itos = {i:ch for i,ch in enumerate(chars)}
    
    encode = lambda x: [stoi[c] for c in x]
    decode = lambda x: "".join([Itos[c] for c in x])
    

    
    

['\n', ' ', '!', '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '°', 'ä', 'æ', 'ë', 'ö', 'ü', '—', '‘', '’', '“', '”']
87


In [18]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)

print(data[:100])

n = int(0.9*len(data))

train_data = data[:n] ## first 90% of the data set is up to train, rest val
val_data = data[n:]

block_size  = 8 
batch_size = 4

torch.manual_seed(1337)

def get_batch(split):
    # generate a small number of data inputs x and targets Y
    
    data = train_data if split == 'train' else val_data
    
    ix = torch.randint(len(data) - block_size, (batch_size,))
    
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    
    
    return x, y 

tensor([69, 57, 54,  1, 37, 67, 58, 56, 58, 63,  1, 64, 55,  1, 41, 65, 54, 52,
        58, 54, 68,  0,  0, 24, 47,  1, 35, 27, 23, 36, 41,  1, 37, 28,  1, 36,
        23, 42, 43, 40, 23, 34,  1, 41, 27, 34, 27, 25, 42, 31, 37, 36,  5,  0,
         0, 37, 40,  1, 42, 30, 27,  0, 38, 40, 27, 41, 27, 40, 44, 23, 42, 31,
        37, 36,  1, 37, 28,  1, 28, 23, 44, 37, 43, 40, 27, 26,  1, 40, 23, 25,
        27, 41,  1, 31, 36,  1, 42, 30, 27,  1])


In [21]:
xb, yb = get_batch('train')
print(xb)
print(yb)

tensor([[ 1, 58, 68,  1, 50, 61, 68, 64],
        [72, 57, 54, 63,  1, 52, 50, 61],
        [54, 67,  1, 71, 50, 67, 58, 54],
        [ 1, 50, 63, 50, 61, 64, 56, 64]])
tensor([[58, 68,  1, 50, 61, 68, 64,  0],
        [57, 54, 63,  1, 52, 50, 61, 61],
        [67,  1, 71, 50, 67, 58, 54, 69],
        [50, 63, 50, 61, 64, 56, 64, 70]])


In [32]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

# b = batch_size
# t = block_size
# c = vocab_Size

class BiogramLanguageModel(nn.Module):
    def __init__(self, vocab_Size):
        super().__init__()
        
        ## each token directly read off the logits for the next token rom a lookup table
        
        self.token_embedding_table = nn.Embedding(vocab_Size, vocab_Size)
        
    def forward(self, idx, targets=None):
        
        #idx and targets are both (b,t ) tensor of intergers 
        
        logits = self.token_embedding_table(idx) ## (b ,t, c)
        
        # pegamos a linha correspondendo na tabela do embedding e organizamos em um tensor de 3 dimensoes, btc 
        
        ## basicamente o scores
        ## oque acontece é que estamos prevendo o proximo caracter na  sequencia, com base apenas em um unico token 
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            
            logits = logits.view(B*T,C) # batch + block, vocalsize twodimensiona tensor
            targets = targets.view(B*T)
            
            
            loss = F.cross_entropy(logits,targets)
        
        return logits, loss
    
    def generate(self, idx, number_of_maxtokens):
        # idx is (B,T) array of indices in the current position 
        
        for _ in range(number_of_maxtokens):
            
            #get the predictions 
            logits, loss = self(idx)
            
            #focus only on the last time step
            logits = logits[:,-1,:] # becomes (B,C)
            
            # apply softmax to get probabilities
            
            probs = F.softmax(logits, dim=1) # B, C
            
            # sample from the distribuition
            idx_next = torch.multinomial(probs, num_samples=1) # b,1
            
            idx = torch.cat((idx, idx_next), dim=1) # (B, T +1)
            
        return idx
    
    
m = BiogramLanguageModel(vocab_Size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), number_of_maxtokens=100)[0].tolist()))

torch.Size([32, 87])
tensor(5.0693, grad_fn=<NllLossBackward0>)

Evhaa?Qn0Döo“OHyR3jx5b-W)rf(gxVrgGqYWdëTeæf—FXd79vsTpuWJFvQ0!Cha sQZEvT1Yy7
I2ug5”Bc?-5d>dYcrm6yipEe
